In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import time
import spacy
import random
from pathlib import Path
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext import data 
import torchtext
from nltk.tokenize.treebank import TreebankWordDetokenizer
import csv
from itertools import islice
import nltk
nltk.download('punkt')

from nltk import word_tokenize,sent_tokenize
from nltk.tokenize import wordpunct_tokenize
import regex as re
import pickle
from collections import Counter
from tqdm import tqdm
import itertools
import math

# Setting device on GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

torch.backends.cudnn.deterministic = True

# Set the random seed manually for reproducibility.
torch.manual_seed(1234)

path = "drive/My Drive/FiQA/"

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

def load_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

def save_pickle(path, data):
    with open(path, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Using device: cuda

Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [0]:
# path = "drive/My Drive/FiQA/"

# # Doc ID to Doc text
# collection = pd.read_csv(path + "FiQA_train_doc_final.tsv", sep="\t")
# collection = collection[['docid', 'doc']]
# collection = collection.sort_values(by=['docid'])

# # Question ID and Question text
# query_df = pd.read_csv(path + "FiQA_train_question_final.tsv", sep="\t")

# queries = query_df[['qid', 'question']]

# all_answers_id = collection['docid'].to_list()
# #print(all_answers_id)

# # Question ID and Answer ID pair
# qid_docid = pd.read_csv(path + "FiQA_train_question_doc_final.tsv", sep="\t")

# qid_docid = qid_docid [['qid', 'docid']]

# # Cleaning data
# empty_docs = []
# empty_id = []

# for index, row in collection.iterrows():
#     if pd.isna(row['doc']):
#         empty_docs.append(row['docid'])
#         empty_id.append(index)

# # print(empty_docs)
# print(empty_id)

# # Remove empty answers
# collection2 = collection.drop(empty_id)

# # Answers Tokenization
# collection2['doc_processed'] = collection2['doc'].apply(pre_process)
# collection2['tokenized_ans'] = collection2.apply(lambda row: wordpunct_tokenize(row['doc_processed']), axis=1)
# collection2['ans_len'] = collection2.apply(lambda row: len(row['tokenized_ans']), axis=1)

# collection2.head(5)

# len(collection2)

# avg_ans_count = collection2['ans_len'].mean()

# print(avg_ans_count)

# # Questions Tokenization
# queries = queries.copy()
# queries['q_processed'] = queries['question'].apply(pre_process)
# queries['tokenized_q'] = queries.apply(lambda row: wordpunct_tokenize(row['q_processed']), axis=1)
# queries['q_len'] = queries.apply(lambda row: len(row['tokenized_q']), axis=1)

# queries.head(5)

# avg_q_count = queries['q_len'].mean()

# print(avg_q_count)

# # collection2.to_csv("processed_ans.csv", index=None)
# # queries.to_csv("processed_q.csv", index=None)

In [0]:
train_qid_rel = load_pickle(path + "data-bert/qid_rel_train_new.pickle")
# valid_qid_rel = load_pickle(path + "qid_rel_valid.pickle")

In [0]:
# qid_rel = {}

# for index, row in qid_docid.iterrows():
    
#     if row['qid'] not in qid_rel:
#         qid_rel[row['qid']] = []
#     qid_rel[row['qid']].append(row['docid'])

In [0]:
# Answer Ranking for each question
# doc_ranking = pd.read_csv(path + "run_train_small.tsv", sep="\t", header=None)
# doc_ranking = doc_ranking.rename(columns={0: 'qid', 1: 'doc_id', 3:'rank'})

# Create dict for query id and ranked candidates
# key: query ids, values: list of 1000 ranked candidates
def load_cands(path):
    qid_ranked_docs = {}

    with open(path,'r') as f:
        for line in f:
            # [qid, doc_id, rank]
            line = line.strip().split('\t')
            qid = int(line[0])
            doc_id = int(line[1])
            rank = int(line[2])
            
            if qid not in qid_ranked_docs:
                # Create a list of size 500 for each query to store the candidates
                candidates = [0]*500
                qid_ranked_docs[qid] = candidates
            qid_ranked_docs[qid][rank-1] = doc_id

    return qid_ranked_docs

In [0]:
train_cands = load_cands(path + "data-bert/cands_train_500.tsv")
# valid_cands = load_cands(path + "data/cands_valid_500.tsv")
# test_cands = load_cands(path + "data/cands_test_500.tsv")

In [8]:
len(train_cands)

6315

In [10]:
take(10, train_qid_rel.items())

[(0, [18850]),
 (2, [308938]),
 (3, [296717, 100764, 314352, 146317]),
 (4, [196463]),
 (5, [69306]),
 (6, [560251, 188530, 564488]),
 (9, [509122, 184698]),
 (11, [596427]),
 (12, [192516, 338700, 158738]),
 (13, [503678])]

In [0]:
# Get train and validation set
def get_data_set(qid_rel, cands, cand_size):
    data_set = []

    for qid, pos_ans_lst in tqdm(qid_rel.items()):
        num_sample = math.floor(cand_size/len(pos_ans_lst))
        for i, cand_lst in cands.items():
            trimed_cand = [x for x in cand_lst if x not in pos_ans_lst]

        # If there is only 1 rel doc
        if num_sample == cand_size:
            for _ in range(cand_size):
                tmp = []
                tmp.append(qid)
                tmp.append(pos_ans_lst[0])
                neg_doc = random.choice(trimed_cand)
                tmp.append(neg_doc)
                data_set.append(tmp)
        else:
            for _ in range(num_sample):
                for j in range(len(pos_ans_lst)):
                    tmp = []
                    tmp.append(qid)
                    tmp.append(pos_ans_lst[j])
                    neg_doc = random.choice(trimed_cand)
                    tmp.append(neg_doc)
                    data_set.append(tmp)
            for k in range(cand_size % len(pos_ans_lst)):
                tmp = []
                tmp.append(qid)
                tmp.append(pos_ans_lst[k])
                neg_doc = random.choice(trimed_cand)
                tmp.append(neg_doc)
                data_set.append(tmp)

    for row in data_set:
        assert len(row) == 3, "Train/Valid set length is incorrect!"

    assert len(data_set) == cand_size*len(cands), "Dataset size is incorrect!"

    return data_set

In [12]:
cand_size = 50

train_set = get_data_set(train_qid_rel, train_cands, cand_size)
# valid_set = get_data_set(valid_qid_rel, valid_cands, cand_size)

100%|██████████| 6315/6315 [22:51<00:00,  5.08it/s]


In [0]:
# with open(path+'data/data_valid_300.pickle', 'wb') as handle:
#     pickle.dump(valid_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(path+'data-bert/data_train_50.pickle', 'wb') as handle:
    pickle.dump(train_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
take(5, train_qid_rel.items())

[(0, [18850]),
 (2, [308938]),
 (3, [296717, 100764, 314352, 146317]),
 (4, [196463]),
 (5, [69306])]

In [0]:
test_qid_rel = load_pickle(path + "qid_rel_test.pickle")
test_cands = load_cands(path + "data/cands_test_500.tsv")

# take(5, test_qid_rel.items())

In [0]:
# Get test set
test_set = []

for qid, docid in tqdm(test_qid_rel.items()):
    for k, v in test_cands.items():
        trimed_cand = [x for x in v if x not in docid]
        cands = random.sample(trimed_cand, 500-len(docid))
        cands.extend(docid)

        if k == qid:
            tmp = []
            tmp.append(qid)
            tmp.append(docid)
            tmp.append(cands)
            test_set.append(tmp)

100%|██████████| 333/333 [00:40<00:00,  8.50it/s]


In [0]:
for row in test_set:
     assert len(row[2]) == 500, "Dataset size is incorrect!"

In [0]:
print(test_set[4][1])

[591357, 556021, 555097]


In [0]:
print(test_set[4][2])

[77564, 111240, 104043, 141579, 591163, 510989, 484424, 124593, 529312, 376898, 412244, 69938, 483437, 99243, 535817, 590209, 40888, 476550, 509266, 166441, 88952, 122986, 205196, 572848, 263648, 570004, 346852, 595152, 558388, 249839, 193592, 449630, 67728, 105828, 21174, 361580, 315363, 417769, 509073, 550925, 47373, 84709, 478807, 163831, 300081, 21468, 127559, 433933, 395957, 467641, 395912, 414527, 175522, 434066, 407547, 288126, 279759, 414932, 336518, 550924, 89161, 464449, 237039, 108001, 261389, 407870, 158965, 338719, 160125, 174196, 304330, 100728, 487890, 452540, 138511, 400291, 305907, 592596, 18844, 104793, 532804, 1897, 473605, 101750, 52441, 29140, 346495, 446647, 422182, 593554, 95715, 113558, 435320, 385343, 164801, 71097, 277563, 289768, 237107, 296717, 48866, 35267, 120554, 579604, 72168, 401587, 341914, 213331, 323389, 264565, 575669, 203889, 334061, 355344, 200425, 199508, 162653, 336045, 154008, 428592, 310559, 277245, 180539, 578357, 585947, 202570, 387547, 9084

In [0]:
with open(path+'data/data_test_500_rel.pickle', 'wb') as handle:
    pickle.dump(test_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
# import time

# neg_ans = {}

# start = time.time()

# for qid, pos_ans_lst in tqdm(qid_rel.items()):
#     for i, cand_lst in qid_ranked_docs.items():
#         trimed_cand = [x for x in cand_lst if x not in pos_ans_lst]
#     neg_ans_lst = random.sample(trimed_cand, 500 - len(pos_ans_lst))
#     neg_ans[qid] = neg_ans_lst

# end = time.time() - start

# print(end)

# neg_ans[0]

In [0]:
# import time

# neg_ans = {}

# start = time.time()

# for qid, pos_ans_lst in tqdm(qid_rel.items()):
#     for i, cand_lst in qid_ranked_docs.items():
#         trimed_cand = [x for x in cand_lst if x not in pos_ans_lst]
#     neg_ans_lst = random.sample(trimed_cand, len(pos_ans_lst))
#     neg_ans[qid] = neg_ans_lst

# end = time.time() - start

# print(end)

# neg_ans[0]

100%|██████████| 6648/6648 [5:01:36<00:00,  2.32s/it]

18096.25180697441


[263751]

In [0]:
# Dump neg ans
with open(path+'neg_ans_1000.pickle', 'wb') as handle:
    pickle.dump(neg_ans, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
# with open(path + 'neg_ans.pickle', 'rb') as f:
#     neg_ans = pickle.load(f)

In [0]:
import time
start = time.time()
train_set = []

for index, row in qid_docid.iterrows():
    for k, v in neg_ans.items():
        if k == row['qid']:
            tmp = []
            tmp.append(row['qid'])
            tmp.append(row['docid'])
            tmp.append(v)
            train_set.append(tmp)

end = time.time() - start

print(end)

731.2214043140411


In [0]:
with open(path+'train_set_1000_un.pickle', 'wb') as handle:
    pickle.dump(train_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
with open('train_set_100.pickle', 'rb') as f:
    train = pickle.load(f)

In [0]:
train[:10]

[[0, 18850, [543898]],
 [1, 14255, [30163]],
 [2, 308938, [236254]],
 [3, 296717, [233248, 268423, 335375, 50250]],
 [3, 100764, [233248, 268423, 335375, 50250]],
 [3, 314352, [233248, 268423, 335375, 50250]],
 [3, 146317, [233248, 268423, 335375, 50250]],
 [4, 196463, [192307]],
 [5, 69306, [583626]],
 [6, 560251, [148208, 543898, 214522]]]

In [0]:
len(train)

17110

In [0]:
test = qid_docid[:]

In [0]:
test_set = []

start = time.time()

for index, row in qid_docid.iterrows():
    for k, v in qid_ranked_docs.items():
        if k == row['qid']:
            tmp = []
            tmp.append(row['qid'])
            tmp.append(row['docid'])
            tmp.append(v)
            test_set.append(tmp)

end = time.time() - start

print(end)

print(test_set[0])